In [ ]:
# Authentication

Make an app and get keys form https://apps.twitter.co.

In [ ]:
import requests
from requests_oauthlib import OAuth1

In [ ]:
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
from donthackme import CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
verify_response = requests.get(url, auth=auth)

In [ ]:
verify_response.json() # These are credentials connected to your account!

# Search

Check the documentation at https://dev.twitter.com/rest/public/search.

In [ ]:
search_url = 'https://api.twitter.com/1.1/search/tweets.json'
params = {'q': 'data science',
          'result_type': 'recent'}
search_response = requests.get(search_url, auth=auth, params=params)

In [ ]:
import json

In [ ]:
print json.dumps(search_response.json(), indent=2)

In [ ]:
search_response.json().keys()

In [ ]:
search_response.json().get('statuses')[0].keys()

In [ ]:
search_response.json().get('statuses')[0].get('user').keys()

In [ ]:
for i in search_response.json().get('statuses', []):
    print i.get('user').get('screen_name'), i.get('text')[:20]

In [ ]:
from collections import Counter
from urlparse import urlparse

In [ ]:
def get_urls(json_result):
    for status in json_result.get('statuses', []):
        for url in status['entities']['urls']:
             yield urlparse(url['expanded_url']).netloc

In [ ]:
list(get_urls(search_response.json()))

In [ ]:
cnt = Counter(get_urls(search_response.json()))
cnt

In [ ]:
cnt.most_common(5)

In [ ]:
json_response = search_response.json()

In [ ]:
next_path = json_response['search_metadata']['next_results']
print next_path

In [ ]:
more_tweets = requests.get(search_url + next_path, auth=auth).json()

In [ ]:
cnt.update(Counter(get_urls(more_tweets)))

In [ ]:
cnt.most_common(5)

# Pagination Strategies

In [ ]:
def tweets(base_url, params, auth):
    result = requests.get(base_url, params=params, auth=auth).json()
    for tweet in result.get('statuses', []):
        yield tweet
    print result.get('search_metadata', {})
    while result.get('search_metadata', {}).get('next_results', None):
        next_url = base_url + result['search_metadata']['next_results']
        print next_url
        result = requests.get(next_url, auth=auth).json()
        for tweet in result.get('statuses', []):
            yield tweet

In [ ]:
def urls(base_url, params, auth):
    for tweet in tweets(base_url, params, auth):
        for url in status['entities']['urls']:
            yield urlparse(url['expanded_url']).netloc

In [ ]:
from itertools import islice

In [ ]:
itercount = Counter(islice(urls(search_url, params, auth), 500))

In [ ]:
itercount.most_common()

In [ ]:
itercount.values()

## Streaming

In [ ]:
from itertools import islice

In [ ]:
r = requests.post('https://stream.twitter.com/1.1/statuses/filter.json',
                  params={'track': '#python'},
                  auth=auth,
                  stream=True)

In [ ]:
r

In [ ]:
print r.headers

In [ ]:
tweets = r.iter_lines()

In [ ]:
for tweet in islice(tweets, 20):
    try:
        tweet_dict = json.loads(tweet)
        print tweet_dict['text']
    except ValueError:
        print "Empty tweet for keep-alive"

In [ ]:
r.close()